In [4]:
tickers = ['ADA-USD','ARB-USD','BNB-USD','BTC-USD','CAKE-USD','DOGE-USD','DOT-USD','ETH-USD','GALA-USD','HBAR-USD','LINK-USD','MINA-USD','OP-USD','SEI-USD','SOL-USD','UNI-USD','XNA-USD','XRP-USD']

In [19]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Fetch S&P 500 symbols from Wikipedia
def fetch_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)[0]
    return table['Symbol'].tolist()

# Clean up ticker symbols
tickers =  fetch_sp500_tickers()

In [32]:
import requests
import pandas as pd

def fetch_mega_cap_tickers():
    url = "https://stockanalysis.com/list/mid-cap-stocks/" #"https://stockanalysis.com/list/small-cap-stocks/" #"https://stockanalysis.com/list/micro-cap-stocks/"  #"https://stockanalysis.com/list/mega-cap-stocks/"
    #https://stockanalysis.com/list/ "https://stockanalysis.com/list/small-cap-stocks/" "https://stockanalysis.com/list/micro-cap-stocks/" 
    
    try:
        # Fetch the HTML content
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse the HTML table
        tables = pd.read_html(response.text)
        
        # The table we want is typically the first one
        mega_cap_df = tables[0]
        
        # Extract the 'Symbol' column
        tickers = mega_cap_df['Symbol'].tolist()
        
        return tickers
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []

# Fetch and print the ticker symbols
tickers = fetch_mega_cap_tickers()
print("Mega-cap stock tickers:")
print(tickers)

Mega-cap stock tickers:
['VFS', 'AOS', 'DSGX', 'PSN', 'ARMK', 'AAON', 'SBS', 'CCK', 'BRBR', 'BNT', 'CHDN', 'ENPH', 'AA', 'CUBE', 'DOX', 'MTSI', 'PNW', 'EXEL', 'WYNN', 'LCID', 'LAD', 'AEG', 'LKQ', 'TPX', 'TTAN', 'LW', 'FRT', 'CRL', 'CFLT', 'BROS', 'DBX', 'WBS', 'FOUR', 'ALSN', 'EHC', 'NIO', 'GTLB', 'GLOB', 'RBC', 'ALTR', 'AIT', 'BIO', 'GNRC', 'BIO.B', 'UWMC', 'PEN', 'U', 'GL', 'GLBE', 'MRNO', 'ORI', 'GAP', 'MASI', 'PAC', 'WAL', 'CACI', 'AYI', 'BLD', 'MLI', 'STN', 'AES', 'VNOM', 'PRI', 'INGR', 'BSAC', 'BILL', 'ITCI', 'APPF', 'WMS', 'REXR', 'KNTK', 'SUM', 'SOUN', 'BRKR', 'CR', 'RGLD', 'HSIC', 'WCC', 'KEP', 'PNFP', 'NYT', 'VNO', 'KNX', 'CFR', 'MKTX', 'FYBR', 'DRS', 'FBIN', 'ROIV', 'FLR', 'SARO', 'CRS', 'HLNE', 'VFC', 'CBSH', 'PLNT', 'ASTS', 'ONTO', 'RRC', 'LTM', 'EXP', 'ALK', 'HQY', 'WING', 'TFX', 'MTCH', 'GKOS', 'BRX', 'WTFC', 'FSV', 'WBA', 'OGE', 'BBWI', 'AGNC', 'GPK', 'CX', 'VERX', 'ASND', 'TTC', 'APA', 'RGEN', 'SKM', 'LSCC', 'DCI', 'CMA', 'GTLS', 'PEGA', 'KD', 'ZION', 'CIB', 'INFA', 'E

/var/folders/9b/ckc59bc918g30hv5rlmd9yv00000gn/T/ipykernel_39589/878724786.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [34]:
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def volume_heatmap(df, length=60, slength=60,
                   threshold_extra_high=4.0,
                   threshold_high=2.5,
                   threshold_medium=1.0,
                   threshold_normal=-0.5):
    """
    Classify volume based on its deviation from the moving average

    Parameters:
    df: DataFrame with 'volume', 'open', 'close' columns
    length: MA Length
    slength: Standard Deviation Length
    threshold_*: Various threshold multipliers

    Returns:
    String indicating volume category
    """

    print(df.columns)

    # Calculate moving average of volume
    mean = df['Volume'].rolling(window=length).mean()

    # Calculate standard deviation of volume
    std = df['Volume'].rolling(window=slength).std()

    # Calculate how many standard deviations volume is from mean
    stdbar = (df['Volume'] - mean) / std

    # Determine if price closed up or down
    direction = df['Close'] > df['Open']

    # Classify volume levels
    def classify_volume(row):
        if pd.isna(row['stdbar']):
            return 'Insufficient Data'

        if row['stdbar'] > threshold_extra_high:
            return 'Extra High Volume' + (' Up' if row['direction'] else ' Down')
        elif row['stdbar'] > threshold_high:
            return 'High Volume' + (' Up' if row['direction'] else ' Down')
        elif row['stdbar'] > threshold_medium:
            return 'Medium Volume' + (' Up' if row['direction'] else ' Down')
        elif row['stdbar'] > threshold_normal:
            return 'Normal Volume' + (' Up' if row['direction'] else ' Down')
        else:
            return 'Low Volume' + (' Up' if row['direction'] else ' Down')

    # Create temporary dataframe for classification
    temp_df = pd.DataFrame({
        'stdbar': stdbar,
        'direction': direction
    })

    return temp_df.apply(classify_volume, axis=1)


# Set date range
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # Use 30 days of data for volume analysis

# Function to get stock data and check volume condition
def check_volume_condition(ticker):
    try:
        #stock = yf.Ticker(ticker)
        #hist = yf.download(tickers=ticker, period='1y', interval='1d')
        hist = yf.download(tickers=ticker, period='1y', interval='1d',prepost=True)

        hist['volume_category'] = volume_heatmap(hist)
        row = hist.iloc[-1]

        if len(hist) > 0:
            latest_volume = row['volume_category']
            avg_volume = row['Volume']

            if row['volume_category'] in  ['Extra High Volume Up', 'High Volume Up']:
                return ticker, latest_volume, avg_volume
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
    
    return None

# Scan stocks
high_volume_stocks = []

for ticker in tickers:
    result = check_volume_condition(ticker)
    if result:
        high_volume_stocks.append(result)

# Print results
print("Stocks trading more than 2x their average volume:")
for ticker, latest_volume, avg_volume in high_volume_stocks:
    print(f"{ticker}: Latest Volume: {latest_volume}, Avg Volume: {avg_volume:.0f} ")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


[*********************100%***********************]  1 of 1 completed


...

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
Stocks trading more than 2x their average volume:
